In [ ]:
import pybullet as bt
import time
import pybullet_data
import eigen as e
import rbdyn as rbd
import sva as s
import numpy as np
from rbdyn.parsers import *

DT = 0.001 #s
MOVEMENT_TIME = 5000 #ms
URDF = "./resource/two_dof.urdf"

def print_multi_body(m):
    '''Helper function to display the contents of a multibody structure
    '''
    print("number of bodies:", m.nrBodies())
    print("number of joints:", m.nrJoints())
    
    print("bodies:")
    for bi, b in enumerate(m.bodies()):
        print("body index:", bi, "body name:", b.name().decode("utf-8"))
    
    print("joints:")
    for ji, j in enumerate(m.joints()):
        print("joint index:", ji, "joint name", j.name().decode("utf-8"))
        
def body_id_from_name(name, bodies):
    '''Gets the body Id from the body name
    @param name The name of the body
    @param bodies The set of bodies provided by the multibody data structure
    @return Id of the body, -1 if not found
    '''
    for bi, b in enumerate(bodies):
        if (b.name().decode("utf-8") == name):
            return bi
    return -1

def sva_to_affine(s):
    '''Converts a spatial transform matrix to a homogeneous transform matrix
    @param s Spatial transform
    @return Homogeneous transform matrix
    '''
    m4d = e.Matrix4d.Identity()
    R = s.rotation().transpose()
    p = s.translation()
    
    for row in range(3):
        for col in range(3):
            m4d.coeff(row, col, R.coeff(row, col))
    for row in range(3):
        m4d.coeff(row, 3, p[row])
        
    return m4d

def inv_spd_edls(mat, sigma_neg, sigma_pos):
    """
    Exponentially damped least squares for inversion from https://doi.org/10.1177/0278364917698748
    """
    pass

def inv_spd_sr(mat, reg=1e-4):
    '''Converts an inverse into an SVD problem with singularity robust inverse
    @param mat Matrix to be inversed
    @param reg Regularization value for the singularity robust inverse
    @return inverse of matrix
    '''
    mat_np = np.array(mat)
    reg_mat = reg * np.identity(np.size(mat_np,0))
    
    return e.MatrixXd(mat_np.transpose().dot(np.linalg.inv(mat_np.dot(mat_np.transpose()) + reg_mat)))

# import robot as global variable
p = from_urdf_file(URDF)
p.mbc.zero(p.mb)
print("[rbdyn] Imported dynamics of " + p.name.decode("utf-8"))

# setup physics
physicsClient = bt.connect(bt.GUI)
bt.setAdditionalSearchPath(pybullet_data.getDataPath())
bt.setRealTimeSimulation(0)
bt.setTimeStep(DT)

# set gravity direction 
# p.mbc.gravity = e.Vector3d(0,0,9.81) # this is the acceleration at base joint for RNEA
# bt.setGravity(0,0,-9.81)
p.mbc.gravity = e.Vector3d(0,0,0) # this is the acceleration at base joint for RNEA
bt.setGravity(0,0,0)

# import robot
plane_id = bt.loadURDF("plane.urdf")
robot_id = bt.loadURDF(URDF, [0,0,1], bt.getQuaternionFromEuler([0,0,0]), 
                      flags=(bt.URDF_USE_INERTIA_FROM_FILE | bt.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS))
n_dof = bt.getNumJoints(robot_id)
joints_id = range(n_dof)

# setup camera
bt.resetDebugVisualizerCamera(5,0,0,[0,0,1.18])

def set_q_in_mbc(q):
    for i in range(0, n_dof):
        p.mbc.q[i+1][0] = q[i]

def set_dq_in_mbc(dq):
    for i in range(0, n_dof):
        p.mbc.alpha[i+1][0] = dq[i]
    
# observer
residual = e.VectorXd([0] * n_dof)
residual_int = e.VectorXd([0] * n_dof)
Ko_joint = e.MatrixXd(np.diag([100]*n_dof))

# shows the link frame defined by <inertial><origin> in urdf, -1 is the first link
com_ee = list(bt.getDynamicsInfo(robot_id, n_dof-1)[3]) # last link
start = [-x for x in com_ee] # move back to origin of link frame
start[0] += 1.0 # move to ee, which is 1m down the length of link
end_x = start.copy()
end_y = start.copy()
end_z = start.copy()
end_x[0] += 0.3
end_y[1] += 0.3
end_z[2] += 0.3
bt.addUserDebugLine(start, end_x, lineColorRGB=[1,0,0], lineWidth=2, parentObjectUniqueId=robot_id, parentLinkIndex=n_dof-1)
bt.addUserDebugLine(start, end_y, lineColorRGB=[0,1,0], lineWidth=2, parentObjectUniqueId=robot_id, parentLinkIndex=n_dof-1)
bt.addUserDebugLine(start, end_z, lineColorRGB=[0,0,1], lineWidth=2, parentObjectUniqueId=robot_id, parentLinkIndex=n_dof-1)

# setup configuration
bt.setJointMotorControlArray(robot_id, joints_id, bt.VELOCITY_CONTROL, forces=[0] * n_dof)
q_home = [0.7, -0.8]
for i in joints_id:
    bt.resetJointState(robot_id, i, targetValue=q_home[i])

# simulate torque control
for step in range(1000):
    joint_states = bt.getJointStates(robot_id, joints_id)
    
    # read state feedback
    q_bullet = [joint_states[i][0] for i in joints_id]
    dq_bullet = [joint_states[i][1] for i in joints_id]
    
    # update states into mbc
    set_q_in_mbc(q_bullet)
    set_dq_in_mbc(dq_bullet)

    ############################### forward kinematics to update model ###############################
    rbd.forwardKinematics(p.mb, p.mbc)
    rbd.forwardVelocity(p.mb, p.mbc)
    
    # convert
    q = e.VectorXd(q_bullet)
    dq = e.VectorXd(dq_bullet)
    
    # pose
    ofsRot = e.Matrix3d.Identity()
    ofsPos = e.Vector3d(1,0,0)
    T1 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * 
                       p.mbc.bodyPosW[body_id_from_name("link_2", p.mb.bodies())])
    
    # select xz coordinates
    S = e.MatrixXd.Zero(2,6)
    S.coeff(0,3,1)
    S.coeff(1,5,1)
    
    # jacobian, dJacobian [angular; linear]
    jac = rbd.Jacobian(p.mb, ("link_2").encode('utf-8'), ofsPos)
    J1_temp = e.MatrixXd(6, p.mb.nrDof())
    dJ1_temp = e.MatrixXd(6, p.mb.nrDof())
    jac.fullJacobian(p.mb, jac.jacobian(p.mb, p.mbc), J1_temp);
    jac.fullJacobian(p.mb, jac.jacobianDot(p.mb, p.mbc), dJ1_temp);
    J1 = S*J1_temp
    dJ1 = S*dJ1_temp
    
    ############################### forward dynamics to update model ###############################
    fd = rbd.ForwardDynamics(p.mb)
        
    # mass matrix
    fd.computeH(p.mb, p.mbc);
    M = fd.H()

    # nonlinear effects
    fd = rbd.ForwardDynamics(p.mb)
    fd.computeC(p.mb, p.mbc);
    h = fd.C()
    
    # coriolis matrix
    cor = rbd.Coriolis(p.mb)
    C = cor.coriolis(p.mb, p.mbc)
    
    # gravity
    set_dq_in_mbc([0] * n_dof)
    rbd.forwardKinematics(p.mb, p.mbc)
    fd = rbd.ForwardDynamics(p.mb)
    fd.computeC(p.mb, p.mbc)
    g = fd.C()
    set_dq_in_mbc(dq_bullet)
    rbd.forwardKinematics(p.mb, p.mbc)
    rbd.forwardVelocity(p.mb, p.mbc)

    # apply force alongside actuator force
    bt.applyExternalForce(robot_id, n_dof-1, (1,1,0), (0,0,0), bt.LINK_FRAME)
    
    # joint space
    err = e.VectorXd(n_dof)
    q_des = [np.pi/4,np.pi/2]
    for i in range(n_dof):
        err[i] = q_des[i]-q.coeff(i)
    tau = M*(10000*err-200*dq)+h

    # momentum observer
    beta = g - C.transpose()*dq
    residual_int += (residual+tau+C.transpose()*dq-g)*DT
    residual = Ko_joint*(M*dq-residual_int)
    Jc = e.MatrixXd(J1)
    JcT = Jc.transpose()
    JcT_inv = e.MatrixXd(np.linalg.pinv(JcT))
    f = JcT_inv*residual
    
    if step%200 == 0:
        print("r={}".format(residual.transpose()))
        print("f={}".format(f.transpose()))

    # command system:
    for i in joints_id:
        bt.setJointMotorControl2(robot_id, i, bt.TORQUE_CONTROL, force=tau[i])
    
    bt.stepSimulation()
    time.sleep(DT)
bt.disconnect()

[rbdyn] Imported dynamics of TwoJointRobot
r=-4483.16 -2515.72
f= 46.6817 -2533.03
r= 1.99749 0.997849
f=  -1.41249 0.00117601


In [40]:
# selection matrix to swap linear and angular components
S = e.MatrixXd(6,6)
S[0,3]=S[1,4]=S[2,5]=1
S[3,0]=S[4,1]=S[5,2]=1

J = S*J1_temp

F = e.Vector6d()
F[0] = 1
F[3] = -1.414

print(J)
# print(S)
print(F)

print(J.transpose()*F)


    -1.41409    -0.706915
-8.56577e-11 -2.31141e-07
-0.000262115    -0.707298
           0            0
          -1           -1
 3.26795e-07  3.26795e-07
     1
     0
     0
-1.414
     0
     0
 -1.41409
-0.706915


In [11]:
import numpy as np
J1 = np.array([[-1.4141, -0.7071],[0, -0.7071]])
print(J1)
F = np.array([[1],[0.7071]])
print(F)

print(np.matmul(J1.transpose(),F))

[[-1.4141 -0.7071]
 [ 0.     -0.7071]]
[[1.    ]
 [0.7071]]
[[-1.4141    ]
 [-1.20709041]]
